In [1]:
import requests
import pandas as pd
    
pdHtml = pd.read_html("http://landtop.com.tw/products.php?types=1",encoding="utf-8")
pdHtml[0]

,0,1,2,3
0,NaN,型號,價格,NaN
1,iPhone XR 64G,$24800,詳情,NaN
2,iPhone XR 128G,$26500,詳情,NaN
3,iPhone XR 256G,$30500,詳情,NaN
4,iPhone xs 64G (a2097),$32990,詳情,NaN
5,iPhone xs 256G (a2097),$37500,詳情,NaN
6,iPhone xs 512G (a2097),$45800,詳情,NaN
7,iPhone xs max 64G (a2101),$35800,詳情,NaN
8,iPhone xs max 256G (a2101),$40800,詳情,NaN
9,iPhone xs max 512G (a2101),$49300,詳情,NaN


In [63]:
import requests
import pandas as pd
import csv

html = requests.get("http://www.twse.com.tw/exchangeReport/MI_INDEX?response=csv&date=20181206&type=ALLBUT0999&_=1544171814819")
# print(html.text)

with open("stock.csv","w",encoding="utf-8-sig") as fp:
    fp.write(html.text)

In [90]:
with open("stock.csv","r",encoding="utf-8-sig") as fp:
    content = list(csv.reader(fp))
li=[]
for i in content:
    if len(i)==17:
        del i[-1]
        li.append(i)
del li[0]
len(li)

1070

In [100]:
li[2][0]

'="0051"'

In [114]:
for i in range(1,len(li)):
    li[i][0].replace('\"','')